In [3]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [6]:
df = pd.read_csv('label_output_copy.csv')
df

,label,image_id,output
0,iption: x-ray chest frontal and lateral cl...,Jemefon_51549626,NaN
1,no acute findings.,Rokiemed_51388893,no definite acute abnormality.
2,cardiomegaly with moderate interstitial edema.,Hayuxrit_52803982,NaN
3,mild left basilar opacity in setting of decrea...,Phathuro_51403270,no definite acute abnormality.
4,no acute cardiopulmonary or osseous abnormality.,Okorued_52838506,no definite acute abnormality.
...,...,...,...
2238,enlarged cardiac silhouette with interstitial ...,Trosoode_52844342,NaN
2239,no acute cardiopulmonary abnormality. previous...,Opirnag_52893284,no definite acute abnormality.
2240,no acute cardiopulmonary findings and no convi...,Shokuja_50802617,NaN
2241,no radiographic evidence of acute cardiopulmon...,Perpopul_50364068,no definite acute abnormality.


In [8]:
df['label'].str.len().mean()

151.95229603209987

In [168]:
diseases = ['edema', 'pneumonia', 'pneumothorax', 'pleural effusion', 'cardiomegaly', 'rib fracture']
for d in diseases:
    count = df['label'].str.contains(d).sum()
    print(f'{d}: {count} occurences.')

edema: 704 occurences.
pneumonia: 301 occurences.
pneumothorax: 218 occurences.
pleural effusion: 505 occurences.
cardiomegaly: 181 occurences.
rib fracture: 18 occurences.


In [10]:
gpt_df = pd.read_csv('gpt_combined.csv')
gpt_df.head()

,label,output,label_pneumothorax,output_pneumothorax,label_pneumonia,output_pneumonia,label_cardiomegaly,output_cardiomegaly,label_edema,output_edema,label_rib_fracture,output_rib_fracture
0,iption: x-ray chest frontal and lateral cl...,NaN,Not Specified,NaN,Not Specified,NaN,Not Specified,NaN,Not Specified,NaN,NaN,NaN
1,no acute findings.,no definite acute abnormality.,Not Specified,Not Specified,Not Specified,Not Specified,Not Specified,Not Specified,Not Specified,Not Specified,NaN,NaN
2,cardiomegaly with moderate interstitial edema.,NaN,Not Specified,NaN,Not Specified,NaN,Not Specified,NaN,Not Specified,NaN,NaN,NaN
3,mild left basilar opacity in setting of decrea...,no definite acute abnormality.,Not Specified,Not Specified,Not Specified,Not Specified,Not Specified,Not Specified,Not Specified,Not Specified,NaN,NaN
4,no acute cardiopulmonary or osseous abnormality.,no definite acute abnormality.,Not Specified,Not Specified,Not Specified,Not Specified,Not Specified,Not Specified,Not Specified,Not Specified,NaN,NaN


In [14]:
gpt_df['label_edema'].value_counts()

label_edema
Not Specified    2199
Mild               16
Moderate            1
Name: count, dtype: int64

#### Edema

In [4]:
edema = pd.read_csv('gpt_edema.csv')
edema

,label,output,label_edema,output_edema,label_edema_corrected,output_edema_corrected
0,iption: x-ray chest frontal and lateral cl...,NaN,No,No,No,No
1,no acute findings.,no definite acute abnormality.,No,No,No,No
2,cardiomegaly with moderate interstitial edema.,NaN,Not Specified,No,Moderate,No
3,mild left basilar opacity in setting of decrea...,no definite acute abnormality.,No,No,No,No
4,no acute cardiopulmonary or osseous abnormality.,no definite acute abnormality.,No,No,No,No
...,...,...,...,...,...,...
2238,enlarged cardiac silhouette with interstitial ...,NaN,Not Specified,No,Mild,No
2239,no acute cardiopulmonary abnormality. previous...,no definite acute abnormality.,No,No,No,No
2240,no acute cardiopulmonary findings and no convi...,NaN,Not Specified,No,Mild,No
2241,no radiographic evidence of acute cardiopulmon...,no definite acute abnormality.,No,No,No,No


In [5]:
for index, row in edema.iterrows():
    if (row['label_edema_corrected'] == 'Mild'):
        print(f"Report: {row['label']}, gpt_label: {row['label_edema_corrected']}")
# edema['label_edema_corrected'].value_counts()
# edema['label'].str.contains('edema').sum()

Report: top-normal cardiac silhouette. calcified aorta.perihilar and lower lung predominant parenchymal opacities favored to represent moderate pulmonary edema. superimposed infection to be excluded clinically.   small right pleural effusion. possible trace left pleural effusion.   the trachea and hila are unremarkable. degenerative changes of the spine., gpt_label: Mild
Report: large right pleural effusion and associated right basal opacity, which may represent underlying atelectasis or consolidation.   interstitial pulmonary edema. cardiomegaly. pt is not ready, gpt_label: Mild
Report: frontal view is essentially nondiagnostic due to technique. the cardiac silhouette appears mildly enlarged. the lateral view shows no gross consolidation or overt pulmonary edema. however, consider repeat study for further evaluation., gpt_label: Mild
Report: retrocardiac opacity may reflect atelectasis versus aspiration/pneumonia.   mild interstitial edema., gpt_label: Mild
Report: moderately enlarged

In [110]:
edema_accuracy = (edema['label_edema_corrected'] == edema['output_edema_corrected']).mean()
print('Edema accuracy: ', edema_accuracy)

Edema accuracy:  0.6843513152028533


#### Pneumonia

In [111]:
pneumonia = pd.read_csv('pneumonia4.csv')
pneumonia

,label,output,label_pneumonia,output_pneumonia
0,iption: x-ray chest frontal and lateral cl...,NaN,No,Unknown
1,no acute findings.,no definite acute abnormality.,No,No
2,cardiomegaly with moderate interstitial edema.,NaN,No,Unknown
3,mild left basilar opacity in setting of decrea...,no definite acute abnormality.,No,No
4,no acute cardiopulmonary or osseous abnormality.,no definite acute abnormality.,No,No
...,...,...,...,...
2238,enlarged cardiac silhouette with interstitial ...,NaN,No,Unknown
2239,no acute cardiopulmonary abnormality. previous...,no definite acute abnormality.,No,No
2240,no acute cardiopulmonary findings and no convi...,NaN,No,Unknown
2241,no radiographic evidence of acute cardiopulmon...,no definite acute abnormality.,No,No


In [113]:
pneumonia['output_pneumonia'].replace('Unknown', 'No', inplace=True)
pneumonia['output_pneumonia'].value_counts()
# pneumonia['label'].str.contains('pneumonia').sum()

output_pneumonia
No          1934
Definite     309
Name: count, dtype: int64

In [116]:
# for index, row in pneumonia.iterrows():
#     if 'pneumonia' in row['label']:
#         print(f"Report: {row['label']}, gpt_label: {row['label_pneumonia']}")
pneumonia['label'].str.contains('pneumonia').sum()

301

In [117]:
pneumonia_accuracy = (pneumonia['label_pneumonia'] == pneumonia['output_pneumonia']).mean()
print('Pneumonia accuracy: ', pneumonia_accuracy)

Pneumonia accuracy:  0.7748551047703968


#### Pneumothorax

In [161]:
pneumothorax = pd.read_csv('pneumothorax.csv')
pneumothorax

,label,image_id,output,label_pneumothorax_corrected,output_pneumothorax_corrected
0,iption: x-ray chest frontal and lateral cl...,Jemefon_51549626,NaN,No,No
1,no acute findings.,Rokiemed_51388893,no definite acute abnormality.,No,No
2,cardiomegaly with moderate interstitial edema.,Hayuxrit_52803982,NaN,No,No
3,mild left basilar opacity in setting of decrea...,Phathuro_51403270,no definite acute abnormality.,No,No
4,no acute cardiopulmonary or osseous abnormality.,Okorued_52838506,no definite acute abnormality.,No,No
...,...,...,...,...,...
2238,enlarged cardiac silhouette with interstitial ...,Trosoode_52844342,NaN,No,No
2239,no acute cardiopulmonary abnormality. previous...,Opirnag_52893284,no definite acute abnormality.,No,No
2240,no acute cardiopulmonary findings and no convi...,Shokuja_50802617,NaN,No,No
2241,no radiographic evidence of acute cardiopulmon...,Perpopul_50364068,no definite acute abnormality.,No,No


In [162]:
pneumothorax['label_pneumothorax_corrected'].value_counts()
# pneumothorax['label'].str.contains('pneumothorax').sum()

label_pneumothorax_corrected
No    2243
Name: count, dtype: int64

In [163]:
# for index, row in pneumothorax.iterrows():
#     if row['accurate_pneumothorax_in_label'] == 'Yes':
#     # if 'pneumothorax' in row['label']:
#         print(f"Report: {row['label']}, gpt_label: {row['accurate_pneumothorax_in_label']}")

In [164]:
(pneumothorax['label_pneumothorax_corrected'] == pneumothorax['output_pneumothorax_corrected']).mean()

1.0

#### Pleural Effusion

In [169]:
pe = pd.read_csv('gpt_pleural_effusion.csv')
pe

,label,image_id,output,label_pe_analysis,output_pe_analysis
0,iption: x-ray chest frontal and lateral cl...,Jemefon_51549626,NaN,No Pleural Effusion,Not Mentioned
1,no acute findings.,Rokiemed_51388893,no definite acute abnormality.,No Pleural Effusion,No Pleural Effusion
2,cardiomegaly with moderate interstitial edema.,Hayuxrit_52803982,NaN,No Pleural Effusion,Not Mentioned
3,mild left basilar opacity in setting of decrea...,Phathuro_51403270,no definite acute abnormality.,No Pleural Effusion,No Pleural Effusion
4,no acute cardiopulmonary or osseous abnormality.,Okorued_52838506,no definite acute abnormality.,No Pleural Effusion,No Pleural Effusion
...,...,...,...,...,...
2238,enlarged cardiac silhouette with interstitial ...,Trosoode_52844342,NaN,"right, left, small, large",Not Mentioned
2239,no acute cardiopulmonary abnormality. previous...,Opirnag_52893284,no definite acute abnormality.,No Pleural Effusion,No Pleural Effusion
2240,no acute cardiopulmonary findings and no convi...,Shokuja_50802617,NaN,No Pleural Effusion,Not Mentioned
2241,no radiographic evidence of acute cardiopulmon...,Perpopul_50364068,no definite acute abnormality.,No Pleural Effusion,No Pleural Effusion


In [170]:
pe['label_pe_analysis'].value_counts()

label_pe_analysis
No Pleural Effusion              1738
no                                 45
bilateral, small                   44
right, left, small, no             26
right, no                          24
                                 ... 
small, large                        1
left, bilateral                     1
small, large, no                    1
left, bilateral, no                 1
right, left, bilateral, large       1
Name: count, Length: 63, dtype: int64

In [174]:
for index, row in pe.iterrows():
    if (row['label_pe_analysis'] == 'right, left, small, no'):
        print(f"Report: {row['label']}, gpt_label: 'right, left, small, no'")

Report: small left pleural effusion. associated left basilar opacity could reflect atelectasis. infection/aspiration to be excluded clinically.   redemonstration of right upper lobe nodule., gpt_label: 'right, left, small, no'
Report: top-normal cardiac silhouette. calcified aorta.perihilar and lower lung predominant parenchymal opacities favored to represent moderate pulmonary edema. superimposed infection to be excluded clinically.   small right pleural effusion. possible trace left pleural effusion.   the trachea and hila are unremarkable. degenerative changes of the spine., gpt_label: 'right, left, small, no'
Report: interval worsening of perihilar interstitial edema.   increased right basal atelectasis versus consolidation. small right pleural effusion and trace left pleural effusion. increased left basal atelectasis.   no other significant interval change., gpt_label: 'right, left, small, no'
Report: removal of the right arm picc.   possible development of pulmonary edema and a s

In [179]:
revised_pe = pd.read_csv('revised_pe.csv')
revised_pe.head()

,label,image_id,output,label_pe_analysis,output_pe_analysis,label_pe_analysis_revised,output_pe_analysis_revised,label_pe_final,output_pe_final
0,iption: x-ray chest frontal and lateral cl...,Jemefon_51549626,NaN,No Pleural Effusion,Not Mentioned,No Pleural Effusion,Not Mentioned,left,No
1,no acute findings.,Rokiemed_51388893,no definite acute abnormality.,No Pleural Effusion,No Pleural Effusion,No Pleural Effusion,No Pleural Effusion,No,No
2,cardiomegaly with moderate interstitial edema.,Hayuxrit_52803982,NaN,No Pleural Effusion,Not Mentioned,No Pleural Effusion,Not Mentioned,No,No
3,mild left basilar opacity in setting of decrea...,Phathuro_51403270,no definite acute abnormality.,No Pleural Effusion,No Pleural Effusion,No Pleural Effusion,No Pleural Effusion,left,No
4,no acute cardiopulmonary or osseous abnormality.,Okorued_52838506,no definite acute abnormality.,No Pleural Effusion,No Pleural Effusion,No Pleural Effusion,No Pleural Effusion,No,No


In [184]:
revised_pe['label_pe_final'].value_counts()

label_pe_final
No                  1134
left                 274
right                184
large                156
small, left          145
bilateral             78
large, left           65
small, bilateral      64
small, right          64
large, right          40
small                 26
large, bilateral      11
medium, left           2
Name: count, dtype: int64

In [186]:
revised_pe['output_pe_final'].value_counts()

output_pe_final
No                  2221
large                 18
bilateral              3
small, bilateral       1
Name: count, dtype: int64

In [187]:
pe_accuracy = (revised_pe['label_pe_final'] == revised_pe['output_pe_final']).mean()
print('Pleural Effusion accuracy: ', pe_accuracy)

Pleural Effusion accuracy:  0.5033437360677664


#### Cardiomegaly

In [189]:
cardiomegaly = pd.read_csv('gpt_cardiomegaly.csv')
cardiomegaly.head()

,label,image_id,output,label_pe_analysis,output_pe_analysis,label_pe_analysis_revised,output_pe_analysis_revised,label_pe_final,output_pe_final,label_cardiomegaly,output_cardiomegaly
0,iption: x-ray chest frontal and lateral cl...,Jemefon_51549626,NaN,No Pleural Effusion,Not Mentioned,No Pleural Effusion,Not Mentioned,left,No,No,No
1,no acute findings.,Rokiemed_51388893,no definite acute abnormality.,No Pleural Effusion,No Pleural Effusion,No Pleural Effusion,No Pleural Effusion,No,No,No,No
2,cardiomegaly with moderate interstitial edema.,Hayuxrit_52803982,NaN,No Pleural Effusion,Not Mentioned,No Pleural Effusion,Not Mentioned,No,No,Mild,No
3,mild left basilar opacity in setting of decrea...,Phathuro_51403270,no definite acute abnormality.,No Pleural Effusion,No Pleural Effusion,No Pleural Effusion,No Pleural Effusion,left,No,No,No
4,no acute cardiopulmonary or osseous abnormality.,Okorued_52838506,no definite acute abnormality.,No Pleural Effusion,No Pleural Effusion,No Pleural Effusion,No Pleural Effusion,No,No,No,No


In [192]:
cardiomegaly['output_cardiomegaly'].value_counts()
# pretty accurate

output_cardiomegaly
No    2243
Name: count, dtype: int64

In [198]:
card_accuracy = (cardiomegaly['label_cardiomegaly'] == cardiomegaly['output_cardiomegaly']).mean()
print('Cardiomegaly accuracy: ', card_accuracy)
# high accuracy bc there's only 181 reports with mentions of it

Cardiomegaly accuracy:  0.9193045028979046


sdsds